# W2 Form Extraction - Deploy Fine-tuned Model on Amazon Bedrock

Deploy the fine-tuned Amazon Nova Lite model using on-demand inferencing and
verify it with a quick inference test.

In this notebook we will:

- Create an on-demand deployment for the custom model
- Wait for the deployment to become active
- Run a sample inference to verify the deployment

**Prerequisite:** Run `03_finetune_on_bedrock.ipynb` first and ensure the job completed.

## Environment Setup

In [ ]:
import json
import warnings
warnings.filterwarnings("ignore")

from util import *

clients = get_aws_clients()
bedrock         = clients["bedrock"]
bedrock_runtime = clients["bedrock_runtime"]
account_id      = clients["account_id"]

In [ ]:
%store -r custom_model_arn
%store -r test_s3_paths

print(f"Custom model ARN: {custom_model_arn}")

## Create On-demand Deployment

In [ ]:
deployment_arn = create_model_deployment(bedrock, custom_model_arn)

if deployment_arn:
    check_deployment_status(bedrock, deployment_arn)

## Wait for Deployment

On-demand deployments typically take around 30 minutes to become active.

In [4]:
if deployment_arn:
    deployment_ready = wait_for_deployment(bedrock, deployment_arn)
    if not deployment_ready:
        print("Deployment did not complete successfully.")

Waiting for deployment (max 60 min)...
Deployment status: Active

Deployment active after 0.0 minutes


## Test Inference

Run a quick inference on the first test sample to verify the deployment.

In [6]:
messages = build_inference_messages(
    test_s3_paths[0]["s3_uri"], account_id, prefill_json=False
)


response = bedrock_runtime.converse(
    modelId=deployment_arn,
    messages=messages,
    inferenceConfig={
        "maxTokens": 2048,
        "temperature": 0.0,
    },
)
prediction_text = response["output"]["message"]["content"][0]["text"]
prediction = parse_json_from_markdown(prediction_text)
prediction

{'employee': {'name': 'Diana Reyes',
  'address': '094 Harris Prairie, Susanville ME 60154-1359',
  'socialSecurityNumber': '053-93-7915'},
 'employer': {'name': 'West and Sons Inc',
  'ein': '38-0226974',
  'address': '0324 Morgan Brook, Port Shawstad KY 78445-9845'},
 'earnings': {'wages': 126589.34,
  'socialSecurityWages': 122867.85,
  'medicareWagesAndTips': 114182.15,
  'federalIncomeTaxWithheld': 43873.99,
  'stateIncomeTax': 9657.36,
  'localWagesTips': 101209.95,
  'localIncomeTax': 14120.87},
 'benefits': {'dependentCareBenefits': 219, 'nonqualifiedPlans': 158},
 'multiStateEmployment': {'HI': {'localWagesTips': 101209.95,
   'localIncomeTax': 14120.87,
   'localityName': 'Ophelia Junction'},
  'WI': {'localWagesTips': 125139.92,
   'localIncomeTax': 23035.86,
   'localityName': 'William Canyon'}}}

## Save Variables

In [ ]:
%store deployment_arn

print(f"Deployment ARN: {deployment_arn}")
print("Variables saved for subsequent notebooks")